# Importação dataset

In [196]:
from google.colab import files

In [197]:
upload = files.upload()

Saving ações.xlsx to ações (1).xlsx


# Importação de bibliotecas

In [198]:
import pandas as pd

import plotly.express as px

# Criação de dataframes

In [199]:
df_principal = pd.read_excel('ações.xlsx', sheet_name="Principal")

In [200]:
df_chat_gpt = pd.read_excel('ações.xlsx', sheet_name="Chat_gpt")

In [201]:
df_total_acoes = pd.read_excel('ações.xlsx', sheet_name="Total_de_acoes")

In [202]:
df_ticker = pd.read_excel('ações.xlsx', sheet_name="Ticker")

# Análise básicas


In [203]:
df_principal.columns

Index(['Ativo', 'Data', 'Último (R$)', 'Var. Dia (%)', 'Var. Sem. (%)',
       'Var. Mês (%)', 'Var. Ano (%)', 'Var. 12M (%)', 'Val. Mín', 'Val. Máx',
       'Volume'],
      dtype='object')

Selecionar apenas as variáveis de interesse

In [204]:
df_principal = df_principal[['Ativo', 'Data', 'Último (R$)', 'Var. Dia (%)']].copy()


Renomear os nome das colunas

In [205]:
df_principal = df_principal.rename(columns={'Último (R$)':'Valor_final', 'Var. Dia (%)':'Var_dia_pct'}).copy()

Transformar a coluna Var_dia_pct em valor númerico decimal

In [206]:
df_principal['Var_pct'] = df_principal['Var_dia_pct']/100

Criar coluna com valor inicial

In [207]:
df_principal['Valor_ini'] = df_principal['Valor_final']/(df_principal['Var_pct']+1)

# Unir tabelas
Criar coluna na df Principal buscando valor na df Total_de_acoes

In [208]:
df_principal = df_principal.merge(df_total_acoes, left_on="Ativo", right_on="Código", how='left')

# Ajuste no df_principal após união dos dataframes
Remover a coluna Código do df_principal

In [209]:
df_principal = df_principal.drop(columns=['Código'])

Renomear a coluna Qtde. Teórica

In [210]:
df_principal = df_principal.rename(columns={'Qtde. Teórica':'Qtde_teorica'})

Criar coluna de Variação em reais

In [211]:
df_principal['Var_reais'] = (df_principal['Valor_final'] - df_principal['Valor_ini']) * df_principal['Qtde_teorica']

Alterar notação científica

In [212]:
pd.options.display.float_format = '{:,.2f}'.format

Alterar tipo de dados da coluna Qtde_teorica

In [213]:
df_principal['Qtde_teorica'] = df_principal['Qtde_teorica'].astype(int)

Criar coluna resultado

In [214]:
df_principal['Resultado'] = df_principal['Var_reais'].apply(lambda x: "Subiu" if x > 0 else ("Desceu" if x < 0 else "Estável"))

# Unir tabelas
Criar coluna na df Principal buscando valor na df Ticker

In [215]:
df_principal = df_principal.merge(df_ticker, left_on='Ativo', right_on="Ticker", how='left')

# Ajuste no df_principal após união dos dataframes
Remover coluna Ticker do df_principal

In [216]:
df_principal = df_principal.drop(columns='Ticker')

Renomear coluna Nome para Nome_empresa

In [217]:
df_principal = df_principal.rename(columns={'Nome':'Nome_empresa'})

# Unir tabelas
Criar coluna na df Principal buscando valor na df Chat_gpt

In [218]:
df_principal = df_principal.merge(df_chat_gpt, left_on="Nome_empresa", right_on="Nome da Empresa", how='left')

# Ajuste no df_principal após união dos dataframes
Remover coluna Nome da Empresa do df_principal

In [219]:
df_principal = df_principal.drop(columns='Nome da Empresa')

Criar coluna com o agrupamento de idade
  - Menos de 50 anos
  - Mais de 100 anos
  - Entre 50 e 100 anos

In [220]:
df_principal['Cat_idade'] = df_principal['Idade'].apply(lambda x: "Menos de 50 anos" if x <= 50 else("Entre 50 e 100 anos" if (x > 50 and x < 100) else "Mais de 100 anos"))

In [241]:
df_principal

,Ativo,Data,Valor_final,Var_dia_pct,Var_pct,Valor_ini,Qtde_teorica,Var_reais,Resultado,Nome_empresa,Segmento,Inicio_Op,Idade,Cat_idade
0,USIM5,2024-01-26,9.50,5.20,0.05,9.03,515117391,"241,889,725.43",Subiu,Usiminas,Siderurgia,1954,60,Entre 50 e 100 anos
1,CMIN3,2024-01-26,6.82,2.40,0.02,6.66,1110559345,"177,515,970.30",Subiu,CSN Mineração,Mineração,2006,8,Menos de 50 anos
2,PETR3,2024-01-26,41.96,2.19,0.02,41.06,2379877655,"2,140,059,393.53",Subiu,Petrobras,Petróleo,1945,69,Entre 50 e 100 anos
3,SUZB3,2024-01-26,52.91,2.04,0.02,51.85,683452836,"722,946,282.71",Subiu,Suzano,Papel e Celulose,1920,94,Entre 50 e 100 anos
4,CPFE3,2024-01-26,37.10,2.03,0.02,36.36,187732538,"138,573,955.06",Subiu,CPFL Energia,Energia,1905,109,Mais de 100 anos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,CRFB3,2024-01-26,10.71,-2.45,-0.02,10.98,533990587,"-143,635,530.57",Desceu,Carrefour Brasil,Varejo,1977,37,Menos de 50 anos
77,BHIA3,2024-01-26,8.70,-2.46,-0.02,8.92,94843047,"-20,810,240.84",Desceu,Casas Bahia,Varejo,1919,95,Entre 50 e 100 anos
78,RENT3,2024-01-26,56.24,-3.63,-0.04,58.36,853202347,"-1,807,432,634.46",Desceu,Localiza,Alugule de Carros,1965,49,Menos de 50 anos
79,CVCB3,2024-01-26,3.07,-4.36,-0.04,3.21,525582771,"-73,557,408.06",Desceu,CVC,Turismo,1964,50,Menos de 50 anos


# Análise descritiva
- Maior variação
- Menor variação
- Média de variação
- Média de variação que subiram
- Média de variação que desceram

In [221]:
maior = df_principal['Var_reais'].max()
menor = df_principal['Var_reais'].min()
media = df_principal['Var_reais'].mean()
media_subiu = df_principal[df_principal['Resultado'] == "Subiu"]['Var_reais'].mean()
media_desceu = df_principal[df_principal['Resultado'] == "Desceu"]['Var_reais'].mean()

In [222]:
print('Maior variação do dia: {:,.2f}'.format(maior))
#print(f"Maior: {maior:,.2f}")
print('Menor variação do dia: {:,.2f}'.format(menor))
print('Média geral da variação do dia: {:,.2f}'.format(media))
print('Média geral da variação que subiram no dia: {:,.2f}'.format(media_subiu))
print('Média geral da variação que desceram no dia: {:,.2f}'.format(media_desceu))

Maior variação do dia: 4,762,926,995.25
Menor variação do dia: -1,807,432,634.46
Média geral da variação do dia: 165,190,210.48
Média geral da variação que subiram no dia: 448,164,250.23
Média geral da variação que desceram no dia: -181,109,141.75


In [223]:
maior_query = df_principal[df_principal['Var_reais'] == df_principal['Var_reais'].max()]


In [224]:
menor_query = df_principal[df_principal['Var_reais'] == df_principal['Var_reais'].min()]

In [225]:
#fig2
df_resultado_final = pd.concat([maior_query, menor_query], axis=0)
df_resultado_final.reset_index(drop=True, inplace=True)

# Análise qualitativa

Quais são os agentes que se destacaram em relação a variação que subiu?

In [226]:
#fig4
df_principal_subiu = df_principal[df_principal['Resultado'] == "Subiu"]
df_principal_subiu = df_principal_subiu.groupby('Segmento')['Var_reais'].sum().reset_index().sort_values('Var_reais', ascending=False)

Quais são os agentes que se destacaram em relação a variação que desceu?

In [227]:
#fig5
df_principal_desceu = df_principal[df_principal['Resultado'] == "Desceu"]
df_principal_desceu = df_principal_desceu.groupby('Segmento')['Var_reais'].sum().reset_index().sort_values('Var_reais')

Quais foram as variações totais por segmeto?

In [228]:
#fig3
df_analise_segmento = df_principal.groupby('Segmento')['Var_reais'].sum().reset_index()

Qual o balanço do dia em relação aos agentes que tiveram suas variações que subiram e que desceram?

In [229]:
#fig1
df_analise_saldo = df_principal.groupby('Resultado')['Var_reais'].sum().reset_index()

Qual o impacto da idade da empresa, e qual a faixa representa maior sensibilidade?

In [230]:
#fig6 e fig7
df_faixa_idade = df_principal.groupby('Cat_idade').agg({'Var_reais':'sum', 'Cat_idade':'size'}).rename(columns={'Cat_idade': 'Qtde_Empresas'}).reset_index().sort_values('Qtde_Empresas', ascending=False)


# Visualização

- Fig1 = Gráfico de colunas para comparação de variações que subiram, estabilizadas e desceram
- Fig2 = Gráfico de colunas para análise descritiva: Variação versus Empresa (maior e menor variação no dia)
- Fig3 = Gráfico de rosca hierárquico para variações geral: Variação versus Segmento
- Fig4 = Gráfico de barras para variações positivas: Variação versus Segmento
- Fig5 = Gráfico de barras para variações negativas: Variação versus Segmento
- Fig6 = Gráfico de áreas para comparação de variações versus faixa de Idade da Empresa
- Fig7 = Gráfico de colunas para comparação de Qtde de Empresa versus a faixa de Idade da Empresa


In [231]:
# Criando uma nova coluna no DataFrame para indicar se o valor é positivo ou negativo
df_analise_saldo['Tipo'] = df_analise_saldo['Var_reais'].apply(lambda x: 'Negativo' if x < 0 else 'Positivo')

# Definindo as cores para valores negativos e positivos
cores = {'Negativo': 'red', 'Positivo': 'green'}

fig1 = px.bar(df_analise_saldo, x='Resultado', y='Var_reais', title='Variação em reais por Resultado', text='Var_reais', color='Tipo', color_discrete_map=cores)
fig1.update_traces(texttemplate='%{text:,.2f}', textposition='outside')
fig1.show()

In [232]:
# Criando uma nova coluna no DataFrame para indicar se o valor é positivo ou negativo
df_resultado_final['Tipo'] = df_resultado_final['Var_reais'].apply(lambda x: 'Negativo' if x < 0 else 'Positivo')

# Definindo as cores para valores negativos e positivos
cores = {'Negativo': 'red', 'Positivo': 'green'}

fig2 = px.bar(df_resultado_final, x='Nome_empresa', y='Var_reais', title='Maior e Menor Variação em reais por Empresa', text='Var_reais', color='Tipo', color_discrete_map=cores)
fig2.update_traces(texttemplate='%{text:,.2f}', textposition='outside')
fig2.show()

In [240]:
# Formatando os valores para exibir com duas casas decimais e separador de milhar
df_analise_segmento['Var_reais_formatado'] = df_analise_segmento['Var_reais'].apply(lambda x: f'R${x:,.2f}')

# Calculando as porcentagens
total = df_analise_segmento['Var_reais'].sum()
df_analise_segmento['Porcentagem'] = (df_analise_segmento['Var_reais'] / total) * 100


fig3 = px.sunburst(df_analise_segmento, color='Var_reais_formatado', values='Porcentagem', path=['Segmento', 'Var_reais_formatado'], hover_name="Var_reais", height=700, title='Variações por Segmento')
fig3.update_traces(hovertemplate='<b>%{label}:</b> R$ %{value:,.2f}(%{percentRoot:.0%})')
fig3.show()


In [234]:
fig4 = px.bar(df_principal_subiu[::-1], x='Var_reais', y='Segmento', orientation='h')
fig4.update_layout(
    title='Variações que subiram por Segmento',
    xaxis_title='Variação em reais',
    yaxis_title='Segmento'
)
fig4.show()

In [235]:
fig5 = px.bar(df_principal_desceu[::-1], x='Var_reais', y='Segmento', orientation='h')
fig5.update_layout(
    title='Variações que desceram por Segmento',
    xaxis_title='Variação em reais',
    yaxis_title='Segmento'
)
fig5.show()

In [236]:
# Criando uma nova coluna no DataFrame para indicar se o valor é positivo ou negativo
df_faixa_idade['Tipo'] = df_faixa_idade['Var_reais'].apply(lambda x: 'Negativo' if x < 0 else 'Positivo')

# Definindo as cores para valores negativos e positivos
cores = {'Negativo': 'red', 'Positivo': 'green'}

fig6 = px.bar(df_faixa_idade, x='Cat_idade', y='Var_reais', labels={'x':'Faixa de idade', 'y':'Variação em reais'}, title='Variações versus faixa de Idade da Empresa', text='Var_reais', color='Tipo', color_discrete_map=cores)
fig6.update_traces(texttemplate='%{text:,.2f}', textposition='outside')
fig6.show()

In [237]:
fig7 = px.bar(df_faixa_idade, x='Cat_idade', y='Qtde_Empresas', labels={'x':'Faixa de idade', 'y':'Variação em reais'}, title=' Quantidade de empresas por faixa de idade', text='Qtde_Empresas')
fig7.update_traces(texttemplate='%{text:,.2f}', textposition='outside')
fig7.show()